# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [154]:
#Загрузим необходимые библиотеки
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords 
from tqdm.notebook import tqdm
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import catboost as cb
from nltk.corpus import wordnet


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ARTEM\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Подготовка

In [155]:
#Загрузим данные, добавим Index_col=0, потому что подгружалась дополнительный столбец
try: 
    data = pd.read_csv('/datasets/toxic_comments.csv',index_col=0)
except:
    data=pd.read_csv('C:/Users/ARTEM/Desktop/Project_DS/Project Ds/toxic_comments.csv',index_col=0)

In [156]:
#Посмотрим на данные
data.head(5)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [157]:
#Посмотрим на данные с этого среза
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [158]:
#Создадим корпус постов
corpus = list(data['text'])

In [159]:
#Инициализируем лемматайзер
lemmatizer = WordNetLemmatizer()


In [160]:
def clear_text(text):
    y=re.sub(r"[^'a-zA-Z ]", ' ', text) 
    clear=" ".join(y.split())
    return clear

In [161]:
#Найдем правильный POS-тег
from nltk.corpus import wordnet
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [162]:
sentence = "The striped bats are hanging on their feet for best"
print([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])

['The', 'strip', 'bat', 'be', 'hang', 'on', 'their', 'foot', 'for', 'best']


In [164]:
%%time
def transform(text):
    a=[]
    for i in nltk.word_tokenize(text):
        b=lemmatizer.lemmatize(i, get_wordnet_pos(i))
        a.append(b)
    return ' '.join(a) 

ready=[]
for i in tqdm(range(len(corpus))):    
    ready.append(transform(clear_text(corpus[i])))
data['lemm_text']=pd.Series(ready, index=data.index)

  0%|          | 0/159292 [00:00<?, ?it/s]

CPU times: total: 1h 18min 4s
Wall time: 1h 17min 59s


In [167]:
#Проверим
data

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww He match this background colour I 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I 'm really not try to edit war It 's ...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...
...,...,...,...
159446,""":::::And for the second time of asking, when ...",0,And for the second time of ask when your view ...
159447,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That be a ho...
159448,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm there no actual article for prosti...
159449,And it looks like it was actually you who put ...,0,And it look like it be actually you who put on...


In [171]:
#Разделим данные на признак и таргет
features = data['lemm_text']
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [172]:
#Проверим, все ок!
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(119469,)
(39823,)
(119469,)
(39823,)


In [173]:
#Векторизируем данные
nltk.download('stopwords') 
stop_words = set(stopwords.words('english'))
count_vect = TfidfVectorizer (stop_words=stop_words) 
features_train=count_vect.fit_transform(features_train)
features_test=count_vect.transform(features_test)
print("Размер мешка с учётом стоп-слов:", features_train.shape)
print("Размер мешка с учётом стоп-слов:", features_test.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ARTEM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Размер мешка с учётом стоп-слов: (119469, 134044)
Размер мешка с учётом стоп-слов: (39823, 134044)


#### Вывод
- Создан корпус постов и преобразован столбец 'text' в список текстов
- Тексты переведены в стандартный для Python формат: кодировку Unicode U
- Тексты лемматизированы с помощью WordNetLemmatizer и очищены
- Произведена векторизация текстов с помощью CountVectorizer, произведены очистка от ненужных слов
- Данные разбиты на обучающую и тестовую выборки 1:4

## Обучение

In [174]:
#Обучим логистическую регрессию
model = LogisticRegression(random_state=12345,solver='liblinear')
param_search={'penalty':["l1","l2"],'max_iter':[100,1000,2500,5000]}
gsearch = GridSearchCV(estimator=model, cv=3, param_grid=param_search, scoring = 'f1')
gsearch.fit(features_train, target_train)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_

In [175]:
print(best_score)
print(best_model)

0.7491350642998696
LogisticRegression(penalty='l1', random_state=12345, solver='liblinear')


In [176]:
#Обучим случайный лес
model = RandomForestClassifier(random_state=12345)
param_search = { 
    'n_estimators': [3, 10, 30],
    'max_depth' : [i for i in range(1,5)]
}
gsearch = GridSearchCV(estimator=model, cv=3, param_grid=param_search, scoring = 'f1')
gsearch.fit(features_train, target_train)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_

In [177]:
print(best_score)
print(best_model)

0.0006585175425755136
RandomForestClassifier(max_depth=4, n_estimators=3, random_state=12345)


In [178]:
#Обучем кэтбуст
model = cb.CatBoostClassifier(iterations=5,random_state=12345)
#Определяю словарь с набором параметров
param_search = {'depth': [1, 5]}

gsearch = GridSearchCV(estimator=model, cv=3,param_grid=param_search, scoring ='f1')
gsearch.fit(features_train, target_train)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_
best_param=gsearch.best_params_

Learning rate set to 0.5
0:	learn: 0.3557218	total: 232ms	remaining: 929ms
1:	learn: 0.2939784	total: 298ms	remaining: 447ms
2:	learn: 0.2743006	total: 363ms	remaining: 242ms
3:	learn: 0.2661671	total: 429ms	remaining: 107ms
4:	learn: 0.2602970	total: 497ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3582325	total: 69.5ms	remaining: 278ms
1:	learn: 0.2930260	total: 140ms	remaining: 209ms
2:	learn: 0.2745811	total: 209ms	remaining: 140ms
3:	learn: 0.2669147	total: 282ms	remaining: 70.4ms
4:	learn: 0.2616268	total: 355ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3554457	total: 70.5ms	remaining: 282ms
1:	learn: 0.2934114	total: 142ms	remaining: 213ms
2:	learn: 0.2733860	total: 212ms	remaining: 141ms
3:	learn: 0.2654239	total: 284ms	remaining: 70.9ms
4:	learn: 0.2602592	total: 356ms	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.3547895	total: 326ms	remaining: 1.3s
1:	learn: 0.2678432	total: 682ms	remaining: 1.02s
2:	learn: 0.2417908	total: 1.04s	remaining: 691ms
3:	

In [179]:
print(best_score)
print(best_model)
print(gsearch.best_params_)

0.5340327192018443
{'depth': 5}


In [181]:
#Сведем результаты
result = {'Модель': ['Логистическая регрессия', 'Случайный лес', 'Catboost'],
                  'f1-score': [0.75, 0.0006, 0.53]
                 }
result=pd.DataFrame(result)


result.style.hide_index()

C:\Users\ARTEM\AppData\Local\Temp\ipykernel_23500\464169241.py:8: FutureWarning: this method is deprecated in favour of `Styler.hide(axis='index')`
  result.style.hide_index()


Модель,f1-score
Логистическая регрессия,0.750000
Случайный лес,0.000600
Catboost,0.530000


#### Вывод
Применили 3 модели из них наиболее оптимальным выглядит логистическая регрессия. Были попытки в catboost добавить дополнительные параметры, но, к сожалению, слишком сильно увеличивается время работы

## Выводы

In [182]:
#Проверим логистическую регрессию на тестовой выборке
model = LogisticRegression(penalty='l1', random_state=12345, solver='liblinear')
model = model.fit(features_train, target_train)
predict = model.predict(features_test)
print(f1_score(target_test,predict))



0.7780744905130006


#### Вывод
По результатам теста значение метрики F1 Score не меньше 0.75, мы в целевых значениях!